# LangChain: LLM Chain

LLM Chain 是 Langchain 中的一個重要組件，旨在將語言模型（LLM，Large Language Model）與一系列步驟（例如提示設計、輸入處理、後處理等）組合在一起，實現複雜的工作流程。簡單來說，LLM Chain 就是將語言模型（如 GPT）和不同的操作鏈接起來，讓用戶能夠根據特定需求進行更靈活的操作。

In [ ]:
!pip install -q --upgrade langchain
# 安裝langchain + google or openai
!pip install -q "langchain[google-genai]"
!pip install -q langchain-classic

from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import (
    JsonOutputParser,
    CommaSeparatedListOutputParser,
)
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import (
    PromptTemplate, # 提示詞模板
    ChatPromptTemplate,
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_classic.chains import (
    LLMChain,
    SimpleSequentialChain,
    RouterChain,
)
# 用於在對話中運行自定義邏輯。例如，你可以使用它來處理用戶輸入前的一些預處理操作，或者在生成回應後進行一些後處理操作。
from langchain_core.runnables import RunnableLambda
from google.colab import userdata
from rich.pretty import pprint
import os

# 設定API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
# 建立模型
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

### LLMChain

In [ ]:
prompt_tpl = PromptTemplate.from_template(
    '請給我講1個關於{type}的笑話'
)

# 創建 LLMChain，將模型和提示模板結合在一起
chain = LLMChain(llm=model, prompt=prompt_tpl)

# 使用 LLMChain 執行，並傳入 '軟體工程師' 作為參數
pprint(chain.invoke('軟體工程師'))

In [ ]:
prompt_tpl = PromptTemplate.from_template(
    '請給我講1個關於{type}的笑話'
)
chain = LLMChain(
    llm=model,
    prompt=prompt_tpl,
    output_key='content' # 輸出的內容會存儲在 'content' 中，默認為 'text'
)

pprint(chain.invoke('軟體工程師'))

In [ ]:
prompt_tpl = PromptTemplate.from_template(
    '請給我講{count}個關於{type}的笑話'
)

# 創建 LLMChain，將語言模型和提示模板結合
chain = LLMChain(llm=model, prompt=prompt_tpl)

# 使用 invoke 方法，傳遞參數 'count' 和 'type'，來生成笑話
# 這裡要求語言模型講 2 個關於程式員的笑話
print(chain.invoke({'count': 2, 'type': '軟體工程師'}))

# 使用 predict 方法，傳遞參數 'count' 和 'type'，來生成笑話
# 這裡要求語言模型講 1 個關於程式員的笑話
# print(chain.predict(count=1, type='軟體工程師'))

加入output Parser

In [ ]:
output_parser.get_format_instructions()

In [ ]:
# 初始化 CommaSeparatedListOutputParser，用來解析逗號分隔的清單輸出
output_parser = CommaSeparatedListOutputParser()

# 獲取格式指引，這將告訴我們如何格式化輸出的結果
instructions = output_parser.get_format_instructions()

# 定義提示模板，將會用來生成請求的問題
prompt_tpl = PromptTemplate(
    template='請舉例3個最具有代表性的{type}名稱\n{instructions}',  # 請求輸入變數並插入格式化指引
    input_variables=['type'],  # 定義輸入變數 'type'
    partial_variables={'instructions': instructions}  # 設定部分變數，這裡是格式化指引
)
pprint(prompt_tpl)

# 創建 LLMChain，將語言模型、提示模板和輸出解析器組合起來
chain = LLMChain(
    llm=model,  # 模型
    prompt=prompt_tpl,  # 使用上述定義的提示模板
    output_parser=output_parser  # 使用輸出解析器來處理結果
)

# 執行鏈，並傳入 '程式語言' 作為 'type' 參數
print(chain.invoke('程式語言'))


## LangChain Expression Language (LCEL)

LCEL（LangChain Event-Driven Language Model）是 LangChain 中的一個新型架構，它主要是為了支援事件驅動的語言模型操作而設計的。LCEL 允許開發者建立基於事件驅動的工作流，從而使得應用程式能夠在特定事件發生時觸發相應的語言模型（LLM）處理邏輯。

主要特點：
1. 事件驅動架構：

LCEL 提供了一種方式來根據外部事件或內部條件來驅動語言模型的行為。這樣，系統能夠根據實際情況動態地生成回應，而不是依賴於傳統的請求-回應模式。

2. 動態工作流：

LCEL 可以根據事件來觸發不同的語言模型任務，實現更靈活的自動化處理流程。這讓開發者可以創建高度自訂化的工作流，並在不同場景中根據需求進行即時響應。

3. 擴展性與靈活性：

這樣的架構設計使得 LCEL 特別適用於複雜的應用場景，尤其是那些需要即時反應或高度動態生成文本的情況。例如，聊天機器人、推薦系統或智能客服等。

### 工作原理：
LCEL 通常包括一組事件源，這些事件源可以是任何類型的輸入（例如，使用者的行為、時間觸發或外部 API 回應）。

當某個事件發生時，LCEL 會啟動對應的語言模型處理過程，並且根據事件的性質來生成結果。這些生成的結果可以被用來作為回應、報告或觸發後續的行動。

In [ ]:
# 定義一個聊天提示模板 (ChatPromptTemplate)，其包含系統訊息與使用者輸入
prompt_template = ChatPromptTemplate.from_messages(
    [
        # 系統訊息：告訴模型自己是一位專業的股票大師，會提供股票的精闢見解與分析
        ("system", '你是一位專業的股票大師，對股票提供精闢的見解與分析。'),
        # 使用者訊息：接收使用者的輸入並進行分析
        ('human', '{input}'),
    ]
)

# 基本字串解析器
parser = StrOutputParser()

# 建立 LLM Chain，使用運算子"|", 將聊天提示模板、語言模型（llm）和輸出解析器串接在一起
llm_chain = prompt_template | model | parser

# 使用 LLM Chain 來生成回應
response = llm_chain.invoke({'input': '台積電股票潛力如何？'})

# 輸出回應結果
print(response)

`SimpleSequentialChain`

In [ ]:
# 設定劇本生成的提示模板
script_prompt_tpl = PromptTemplate.from_template("""
    你是一位優秀的編劇。請使用你豐富的想像力根據我設定的標題編寫一個故事概要
    標題:{title}
""")
# 創建劇本鏈，將語言模型和提示模板結合
script_chain = LLMChain(llm=model, prompt=script_prompt_tpl)

# 設定廣告文案生成的提示模板
adv_prompt_tpl = PromptTemplate.from_template("""
    你是一位優秀的廣告文案寫手。請根據我設定的故事概要
    為我的故事寫一段儘可能簡短但要讓人有觀看慾望的廣告詞。
    故事概要:{story}
""")
# 創建廣告鏈，將語言模型和提示模板結合
ad_chain = LLMChain(llm=model, prompt=adv_prompt_tpl)

# 將劇本鏈和廣告鏈串聯起來，形成一個完整的流程
chain = SimpleSequentialChain(
    chains=[script_chain, ad_chain],  # 設定處理順序，先生成劇本，再生成廣告
    verbose=True  # 顯示處理過程中的詳細信息
)

# 輸入標題 "張飛大戰岳飛"，並執行整個鏈流程，生成結果
response = chain.invoke('張飛大戰岳飛')
pprint(response)

### `RouterChain`

Router 能建立未確定的鏈，其中上一個步驟的輸出定義下一步。**舉例來說：假設有兩個針對不同類型問題最佳化的模板，並且希望根據使用者輸入選擇模板。**


**1.Router的功能：**
根據前一個步驟的結果來決定下一步要做什麼。

**2.例子：**
你有兩個不同的模板，分別適合不同類型的問題。
你希望根據用戶輸入自動選擇合適的模板。

**3.具體實現：**
使用 `RunnableBranch`，它會**根據輸入的條件來選擇並運行相應的分支程序**。
如果沒有匹配的條件，它會運行預設的程序。
簡單說，RunnableBranch 就是幫你自動選擇和運行最適合當前情況的流程。

In [ ]:
prompt_template = PromptTemplate.from_template('''
    鑑於下面的用戶問題，將其分類為“數學”、“物理”或“其他”，只要輸出類別
    <Q>
    {question}
    </Q>
''')
classifier_chain = prompt_template | model | StrOutputParser()

In [ ]:
classifier_chain.invoke({'question': '第一個大於 50 的質數，且加 1 可以被 3 整除的數字是多少？'})

In [ ]:
classifier_chain.invoke({'question': '黑洞是甚麼？'})

In [ ]:
classifier_chain.invoke({'question': '全球衛生日是哪一天？'})

定義一個route, 準備建立Expert answer system

In [ ]:
def route(info):
    if '數學' in info['key_topic']:
        return math_prompt
    elif '物理' in info['key_topic']:
        return physics_prompt
    else:
        return general_prompt

# Mathematics
math_template = '''
你是一位非常聰明的數學教授，很擅長回答數學問題。
你能夠將難題分解成各個小問題部分，回答小問題後將它們組合起來回答更廣泛的問題。
問題如下:
{question}'''
math_prompt = PromptTemplate.from_template(math_template)

# Physics
physics_template = '''
你是一位非常聰明的物理學教授，很擅長回答物理問題。
你擅長以簡潔易懂的方式回答有關物理的問題。當你不知道某個問題的答案時，你就承認你不知道。
問題如下:
{question}'''
physics_prompt = PromptTemplate.from_template(physics_template)

# General Assistant
general_template = '''
你是一個聰明的全能助手，請盡可能準確地回答問題。
問題如下:
{question}'''
general_prompt = PromptTemplate.from_template(general_template)

建立router_chain，RunnableLambda(route) 表示將路由函數 route 作為可運行的 Lambda 函數，用於根據條件選擇對應的template

In [ ]:
router_chain = {
    'key_topic': classifier_chain,
    'question': lambda x: x['question']
} | RunnableLambda(route)

In [ ]:
# 測試數學問題
router_chain.invoke({'question': '第一個大於 40 的質數，且加 1 可以被 3 整除的數字是多少？'})

In [ ]:
# 測試物理問題
router_chain.invoke({'question': '黑洞是甚麼？'})

In [ ]:
# 定義了一個llm chain，這個鏈將多個步驟串聯起來，實現對用戶輸入的處理和回應
llm_chain = router_chain | model | StrOutputParser()

In [ ]:
# 測試數學問題
print(llm_chain.invoke({'question': '第一個大於 40 的質數，且加 1 可以被 3 整除的數字是多少？'}))

In [ ]:
# 測試物理問題
print(llm_chain.invoke({'question': '黑洞是甚麼？'}))

In [ ]:
# 測試其他問題
print(llm_chain.invoke({'question': '台灣清明節是哪一天？'}))